In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt

In [4]:
import seaborn as sns

In [5]:
%matplotlib inline

In [6]:
import socket

In [7]:
df = pd.read_excel('Final_Train.xlsx')

In [8]:
df_test = pd.read_excel('Final_Test.xlsx')

In [9]:
big_df = df.append(df_test)

/home/rushikesh/.local/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [10]:
big_df['Qual_1'] = big_df['Qualification'].str.split(',').str[0]
big_df['Qual_2'] = big_df['Qualification'].str.split(',').str[1]
big_df['Qual_3'] = big_df['Qualification'].str.split(',').str[2]
big_df['Qual_1'].fillna('XXX', inplace= True)
big_df['Qual_2'].fillna('XXX', inplace= True)
big_df['Qual_3'].fillna('XXX', inplace= True)

In [11]:
big_df['years_exp'] = big_df['Experience'].str.slice(stop=2).astype(int)

In [12]:
big_df['Rating'].fillna('0%', inplace=True)
big_df['Rating'] = big_df['Rating'].str.slice(stop=1).astype(int)

In [13]:
big_df['City'] = big_df['Place'].str.split(',').str[1]
big_df['Locality'] = big_df['Place'].str.split(',').str[0]
big_df['City'].fillna('XXX', inplace=True)
big_df['Locality'].fillna('XXX', inplace=True)

In [14]:
big_df['Misc'] = big_df['Miscellaneous_Info'].str.split('%').str[0]
big_df['Misc_len'] = big_df['Misc'].str.len()
big_df.loc[big_df['Misc_len']>3, 'Misc'] = 0
big_df['Misc'].fillna(0,inplace = True)
big_df['Misc'] = big_df['Misc'].astype(int)
big_df['Misc_2'] = big_df['Miscellaneous_Info'].str.split('% ').str[1]
big_df['Misc_3'] = big_df['Misc_2'].str.split(' ').str[0]
big_df['Misc_3'].fillna(0,inplace = True)
big_df['Misc_3_len'] = big_df['Misc_3'].str.len()
big_df.loc[big_df['Misc_3_len']>3, 'Misc_3'] = 0
big_df.loc[big_df['Misc_3']==',', 'Misc_3'] = 0
big_df['Misc_3'] = big_df['Misc_3'].astype(int)

In [15]:
big_df = big_df.drop(big_df[['Experience', 'Miscellaneous_Info', 'Place', 'Qualification', 'Misc_len', 'Misc_3_len']], axis =1)
df_train = big_df[0:5961]
df_test = big_df[5961:]
df_test = df_test.drop(['Fees'], axis=1)

In [16]:
from sklearn.preprocessing import LabelEncoder

lb_encode = LabelEncoder()
df_test['Qual_1_code'] = lb_encode.fit_transform(df_test['Qual_1'])
df_test['Qual_2_code'] = lb_encode.fit_transform(df_test['Qual_2'])
df_test['Qual_3_code'] = lb_encode.fit_transform(df_test['Qual_3'])

df_test['Profile_code'] = lb_encode.fit_transform(df_test['Profile'])
df_test['City_code'] = lb_encode.fit_transform(df_test['City'])
df_test['Locality_code'] = lb_encode.fit_transform(df_test['Locality'])

In [17]:
df_test_merge_1 = df_test[['Qual_1', 'Qual_1_code']].drop_duplicates()
df_test_merge_2 = df_test[['Qual_2', 'Qual_2_code']].drop_duplicates()
df_test_merge_3 = df_test[['Qual_3', 'Qual_3_code']].drop_duplicates()
df_test_merge_4 = df_test[['Profile', 'Profile_code']].drop_duplicates()
df_test_merge_5 = df_test[['City', 'City_code']].drop_duplicates()
df_test_merge_6 = df_test[['Locality', 'Locality_code']].drop_duplicates()

In [18]:
df_train = pd.merge(df_train, df_test_merge_1[['Qual_1', 'Qual_1_code']], on='Qual_1', how='left')
df_train = pd.merge(df_train, df_test_merge_2[['Qual_2', 'Qual_2_code']], on='Qual_2', how='left')
df_train = pd.merge(df_train, df_test_merge_3[['Qual_3', 'Qual_3_code']], on='Qual_3', how='left')
df_train = pd.merge(df_train, df_test_merge_4[['Profile', 'Profile_code']], on='Profile', how='left')
df_train = pd.merge(df_train, df_test_merge_5[['City', 'City_code']], on='City', how='left')
df_train = pd.merge(df_train, df_test_merge_6[['Locality', 'Locality_code']], on='Locality', how='left')

In [19]:
df_xgb = df_train[['Qual_1_code', 'Qual_2_code', 'Qual_3_code','years_exp', 'Rating', 'Profile_code', 'City_code', 'Misc', 'Misc_3', 'Fees']]
X = df_xgb.drop(['Fees'], axis=1)
y = df_xgb.Fees

In [27]:
#import xgboost as xgb
#from sklearn.metrics import mean_squared_error
#data_dmatrix = xgb.DMatrix(data=X, label=y)
from sklearn.tree import DecisionTreeRegressor

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=123)

#dtr = DecisionTreeRegressor()
#dtr.fit(X_train, y_train)
xg_reg = xgb.XGBRegressor(objective = 'reg:linear', colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 5, n_estimators = 800)
xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)

In [57]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" %(rmse))

df_test_xgb = df_test[['Qual_1_code', 'Qual_2_code', 'Qual_3_code','years_exp', 'Rating', 'Profile_code', 'City_code', 'Misc', 'Misc_3']]

preds_1 = xg_reg.predict(df_test_xgb)
df_test_xgb['Fees'] = preds_1
df_test_xgb.to_excel('submission.xlsx')

RMSE: 171.335120


/home/rushikesh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
